<a href="https://colab.research.google.com/github/skssushil/Web-mining/blob/master/2_Term_Document_matrix_and_Cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import regex as re
import math
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import files
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
# to search
query =["tutorial on datastructure", "data science", "web mining"," Artificial intelegence"]
url=[]
for i in query:
  q=[]
  for j in search(i, tld="com", num=5, stop=10, pause=2):
    q.append(j)
  url.append(q)
print(url)

[['https://www.tutorialspoint.com/data_structures_algorithms/index.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/data_structures_basics.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/algorithms_basics.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/dsa_useful_resources.htm', 'https://www.tutorialspoint.com/data_structures_algorithms/stack_algorithm.htm', 'https://www.youtube.com/watch?v=RBSGKlAvoiM', 'https://www.geeksforgeeks.org/data-structures/', 'https://www.javatpoint.com/data-structure-tutorial', 'https://www.javatpoint.com/data-structure-interview-questions', 'https://www.javatpoint.com/data-structure-introduction'], ['https://www.kainos.com/my-experience-as-a-data-scientist', 'https://en.wikipedia.org/wiki/Data_science', 'https://en.wikipedia.org/wiki/Master_in_Data_Science', 'https://en.wikipedia.org/wiki/Big_data', 'https://en.wikipedia.org/wiki/Data_analysis', 'https://en.wikipedia.org/wiki/Unstructured_data', 'https://da

In [3]:
f_text=" "

In [4]:
Length=len(url)

In [5]:
Length1=len(url[0])

In [6]:
dataset=[]
for i in range(Length):
  for j in range(Length1):
    req = urllib.request.Request(url[i][j], headers={'User-Agent': 'Mozilla/5.0'})
    data = urllib.request.urlopen(req).read()
    html= data.decode()
    soup = BeautifulSoup(html,features="lxml")
    for script in soup(["script", "style"]):
      script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
  f_text=f_text+text
  dataset.append(f_text)

In [7]:
M=len(dataset)

Pre Processing

In [8]:
def convert_lower_case(data):
  return np.char.lower(data)

In [9]:
def remove_string_special_characters(data):
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', '', data)
    stripped = re.sub('_', '', stripped)

    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)

    # Remove start and end white spaces
    stripped = stripped.strip()
    return stripped.lower()


In [10]:
def stemming(data):
    stemmer= PorterStemmer()    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [11]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [12]:
# Stopword removal
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [13]:
def preprocess(data):
  Clean_data= remove_string_special_characters(data)  # removing special character
  Clean_data=stemming(Clean_data)
  Clean_data= convert_numbers(Clean_data)
  Clean_data=remove_stop_words(Clean_data)
  return Clean_data
# preprocessing of data completed

In [14]:
processed_text1 = []
for i in dataset[:M]:
    processed_text1.append(str(preprocess(i)))

In [15]:
print(len(processed_text1))

4


In [16]:
df = pd.DataFrame({'Doc1': [processed_text1[0]],'Doc2': [processed_text1[1]],'Doc3': [processed_text1[2]],'Doc4': [processed_text1[3]]})

In [17]:
print(df)

                                                Doc1  ...                                               Doc4
0   ds introduct javatpoint home data structur ja...  ...   ds introduct javatpoint home data structur ja...

[1 rows x 4 columns]


In [18]:
# Initialize
vectorizer = TfidfVectorizer()
doc_vec = vectorizer.fit_transform(df.iloc[0])

In [19]:
# Create dataFrame
df2 = pd.DataFrame(doc_vec.toarray().transpose(),
                   index=vectorizer.get_feature_names())

In [20]:
# Change column headers
df2.columns = df.columns

In [21]:
df3=pd.DataFrame(vectorizer.get_feature_names())

In [22]:
print(df2.head(50))

                      Doc1      Doc2      Doc3      Doc4
aapl              0.000000  0.000000  0.000000  0.007846
abil              0.000000  0.000000  0.000000  0.007846
abl               0.000000  0.000000  0.000000  0.007846
abov              0.008472  0.005754  0.004693  0.008189
abstract          0.033890  0.023017  0.018773  0.016378
academi           0.000000  0.000000  0.000000  0.007846
accept            0.000000  0.000000  0.007091  0.006186
access            0.000000  0.014076  0.017222  0.015024
accessori         0.000000  0.049267  0.040185  0.035057
account           0.000000  0.000000  0.007091  0.030930
accru             0.000000  0.000000  0.007091  0.006186
accur             0.000000  0.014076  0.011481  0.010016
achiev            0.000000  0.014076  0.011481  0.015024
acquia            0.000000  0.000000  0.014182  0.012372
across            0.000000  0.007038  0.005741  0.010016
act               0.000000  0.000000  0.000000  0.007846
action            0.000000  0.0

In [23]:
url_list=[*url[0],*url[1],*url[2],*url[3]]

In [24]:
doc=[]
for k in url_list:
  req = urllib.request.Request(k, headers={'User-Agent': 'Mozilla/5.0'})
  data = urllib.request.urlopen(req).read()
  html= data.decode()
  soup = BeautifulSoup(html,features="lxml")
  for script in soup(["script", "style"]):
    script.extract()
  # get text
  text = soup.get_text()
  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split(" "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  doc.append(text)

In [25]:
N=len(doc)

In [26]:
processed_text = []
for i in doc[:N]:
    processed_text.append(word_tokenize(str(preprocess(i))))

In [27]:
len(processed_text)

40

In [28]:
DF = {}
for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [29]:
total_vocab_size = len(DF)
print(total_vocab_size)

9626


In [30]:
total_vocab = [x for x in DF]

In [31]:
print(total_vocab[:20])

['data', 'structur', 'algorithm', 'tutori', 'tutorialspoint', 'home', 'job', 'tool', 'code', 'ground', 'current', 'affair', 'upsc', 'note', 'onlin', 'tutor', 'whiteboard', 'net', 'meet', 'tutorix']


In [32]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [33]:
doc = 0
tf_idf = {}
for i in range(N):    
    tokens = processed_text[i]    
    counter = Counter(tokens)
    words_count = len(tokens)    
    for token in np.unique(tokens):        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))        
        tf_idf[doc, token] = tf*idf
    doc += 1

In [34]:
len(tf_idf)

30472

In [35]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [36]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [37]:
def gen_vector(tokens):
    Q = np.zeros((len(total_vocab)))
    counter = Counter(tokens)
    words_count = len(tokens)
    query_weights = {}
    for token in np.unique(tokens):   
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [38]:
def cosine_similarity(query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print("")
    #print(tokens)
    d_cosines = []
    query_vector = gen_vector(tokens)
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
    return d_cosines

In [39]:
Arr1=np.array(cosine_similarity(query[0]))
Arr2=np.array(cosine_similarity(query[1]))
Arr3=np.array(cosine_similarity(query[2]))
Arr4=np.array(cosine_similarity(query[3]))

In [40]:
Arr1=np.sort(Arr1)[::-1]
Arr2=np.sort(Arr2)[::-1]
Arr3=np.sort(Arr3)[::-1]
Arr4=np.sort(Arr4)[::-1]

In [41]:
print(Arr1)

[0.05897758 0.05779178 0.03642241 0.03534395 0.03203201 0.02679438
 0.02553508 0.02075446 0.01743775 0.01237658 0.01093245 0.00792001
 0.00563358 0.00034619 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [42]:
print(Arr2)

[0.17978494 0.09535605 0.06523546 0.0618833  0.04922685 0.04874491
 0.04784486 0.04350746 0.03336656 0.02994366 0.0272546  0.02410809
 0.02109992 0.02091013 0.02091013 0.02091013 0.02091013 0.02091013
 0.01604274 0.01590294 0.01387858 0.01367625 0.01195706 0.01091454
 0.00998544 0.00967544 0.00953125 0.00914965 0.00909868 0.00825618
 0.00791113 0.00674686 0.00591415 0.00527856 0.00525766 0.0049694
 0.00417058 0.00289409 0.00167538 0.        ]


In [43]:
print(Arr3)

[0.59446764 0.59446764 0.59446764 0.59446764 0.59446764 0.3953234
 0.35475221 0.20773843 0.1928689  0.15600632 0.03983378 0.03046217
 0.0145968  0.01433358 0.01146937 0.01058769 0.01002637 0.00996139
 0.00894279 0.00710005 0.00508511 0.00506153 0.0043481  0.00378828
 0.0036416  0.00290146 0.00234979 0.0018741  0.00181861 0.00178224
 0.00164819 0.0013139  0.00106179 0.001042   0.         0.
 0.         0.         0.         0.        ]


In [44]:
print(Arr4)

[0.29138245 0.28159108 0.22318172 0.14886502 0.14670798 0.12527171
 0.05937472 0.03772473 0.02592894 0.02437975 0.02116451 0.02116451
 0.02116451 0.02116451 0.02116451 0.0192547  0.01022796 0.00767739
 0.00678025 0.00673492 0.00531669 0.00485003 0.00232908 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
